In [1]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
import fastmri
from CLPmodel import *

from fastmri.data.subsample import RandomMaskFunc
from fastmri.data import subsample
from fastmri.data import transforms, mri_data

#from pathlib import Path
#from models.zero_filled.run_zero_filled import save_zero_filled

from fastmri.evaluate import *

In [2]:
#save_zero_filled(Path('mimmo'),Path('zero_filled_img'),'singlecoil',320)

In [3]:
#file_name = 'mimmo/mimmo.h5'
#hf = h5py.File(file_name,'r')
#target = hf['reconstruction_esc'][35]
#kspace = hf['kspace'][35]

mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[8])

def data_transform(kspace, mask, target, data_attributes, filename, slice_num):
    # Transform the data into appropriate format
    
    kspace = transforms.to_tensor(kspace)                      #torch.Size([640, 372, 2])
    masked_kspace, mask = transforms.apply_mask(kspace, mask_func)       #torch.Size([640, 372, 2])
                                                                         #torch.Size([1, 372, 1])
    zero_fill = fastmri.ifft2c(masked_kspace)                  #torch.Size([640, 372, 2])
    
    #crop_zero_fill_abs = fastmri.complex_abs(zero_fill)         #torch.Size([640, 372])
    # ALERT: fare prima inversa e poi il croppe
    
    return zero_fill, masked_kspace, mask, target

dataset = mri_data.SliceDataset(
    root=pathlib.Path('./trainset'),
    transform=data_transform,
    challenge='singlecoil'
)


In [4]:
#file_name = 'zero_filled_img/mimmo.h5'
#hf = h5py.File(file_name,'r')

#ricostruzione_esc2 = hf['reconstruction'][35]
#
#print(ricostruzione_esc2.shape)
#

#fig = plt.figure()
#plt.imshow(pippoabs, cmap='gray')

In [5]:
%%time
import torch.optim as optim

net = Net()

criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.95, amsgrad=False)

psnr = PSNR()
ssim = SSIM()

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    i=0
    
    avg_psnr = 0.0
    avg_ssim = 0.0
    
    for zero_fill, masked_kspace, mask, target in dataset:
            
        input1 = zero_fill.unsqueeze(0).unsqueeze(0)
        input2 = masked_kspace.unsqueeze(0).unsqueeze(0)

        #input1.requires_grad=True
        #input2.requires_grad=True

        outputs = net(input1, input2, mask)

        abs1 = fastmri.complex_abs(outputs[0][0])
        abs2 = transforms.to_tensor(target)

        loss = criterion(abs1, abs2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()

        avg_psnr += psnr(abs1,abs2)
        avg_ssim += ssim(abs1,abs2)

        #if i==35:
        #    fig = plt.figure()
        #    plt.imshow(abs1.detach().numpy(), cmap='gray')
        #    plt.savefig('res/epoch'+str(epoch+1)+'.jpg')
        #    plt.close()
        i+=1
        
        pass
    
    print('Epoch n° (%d) loss: %.3f' % (epoch + 1, running_loss))
    
    print("Average psnr: ", avg_psnr /36)
    print("Average ssim: ", avg_ssim /36)
    
print('Finished Training')

Epoch n° (1) loss: 0.000
Average psnr:  tensor([16.7801])
Average ssim:  tensor([0.2080])
Epoch n° (2) loss: 0.000
Average psnr:  tensor([16.6630])
Average ssim:  tensor([0.2108])
Epoch n° (3) loss: 0.000
Average psnr:  tensor([17.3896])
Average ssim:  tensor([0.2009])
Epoch n° (4) loss: 0.000
Average psnr:  tensor([16.1167])
Average ssim:  tensor([0.1493])
Epoch n° (5) loss: 0.000
Average psnr:  tensor([16.6604])
Average ssim:  tensor([0.1887])
Epoch n° (6) loss: 0.000
Average psnr:  tensor([15.9199])
Average ssim:  tensor([0.1642])
Epoch n° (7) loss: 0.000
Average psnr:  tensor([16.3366])
Average ssim:  tensor([0.1761])
Epoch n° (8) loss: 0.000
Average psnr:  tensor([15.8292])
Average ssim:  tensor([0.1392])
Epoch n° (9) loss: 0.000
Average psnr:  tensor([16.5499])
Average ssim:  tensor([0.1864])
Epoch n° (10) loss: 0.000
Average psnr:  tensor([16.8928])
Average ssim:  tensor([0.1964])
Finished Training
CPU times: user 9min 58s, sys: 1min 21s, total: 11min 20s
Wall time: 2min 52s


In [6]:
PATH = './saved_net.pth'
torch.save(net.state_dict(), PATH)